In [ ]:
import pandas as pd
from collections import defaultdict, Counter
from tqdm import tqdm


Below, we declare some constants used for data collection.

<b>PATH_TO_DATA</b>:<br/>
Replace this with the path to the data files, which can be downloaded from here: https://github.com/danmorse314/hockeyR-data/tree/main/data

<b>Strengths</b>:
- We declare the 9 possible on-ice strengths possible in an NHL game
- Each team may have up to 5 skaters and no less than 3

Note: during delayed penalties or end-game situations, teams may pull their goalie for an extra attacker, resulting in 6 skaters on the ice. Since the pbp data is not consistent enough with these situations, I have decided to not include strengths of 6 skaters.

<b>Metrics</b>:
- TOI: Time on ice (in seconds) for a given team's strength
- GF: Goals for
- GA: Goals allowed

<b>Columns</b>:<br/>
The columns for the output dataframe after scraping the desired seasons are complete.

In [ ]:
# Constants
PATH_TO_DATA = "./hockeyR_data/"

STRENGTHS = [(5,5), (5,4), (4,5), (4,4), (3,3), (4,3), (3,4), (5,3), (3,5)]
METRICS = ['TOI', 'GF', 'GA']
COLUMNS = ['season', 'team', 'strength', 'TOI', 'goals', 'goals_against']

pbp = pd.read_csv(PATH_TO_DATA + "play_by_play_2022_23_lite.csv")
ALL_TEAMS = list(pbp.loc[:, 'away_name'].unique())
ALL_TEAMS

In [ ]:
# this class is used for preprocessing batch penalties
# once the batch of penalties are resolved, the normal Penalty class should be used for the clock
class PrePenalty:
    def __init__(self, time_length = 120):
        self.time_length = time_length
        
# once we process batch penalties, use these objects to add to the penalty clock
class Penalty:
    def __init__(self, time_start, time_length = 120, severity = 'Minor', 
                 is_4v4 = False, remaining_majors = 0, remaining_minors = 0):
        self.time_start = time_start
        self.time_length = time_length
        self.severity = severity
        # if is_4v4 True, minor is of the 4v4 type. both teams down a man;
        # even though this is on the clock, goals don't expire them
        self.is_4v4 = is_4v4
        # used when a player has multiple penalties i.e. a double minor (2+2) or a 5+2
        self.remaining_majors = remaining_majors
        self.remaining_minors = remaining_minors
        
    def __str__(self):
        period = int(self.time_start / 1200) + 1
        time = self.time_start - (period - 1) * 1200
        return f'{self.severity} Penalty at {time} P{period}'
    
    def __repr__(self):
        period = int(self.time_start / 1200) + 1
        time = self.time_start - (period - 1) * 1200
        return f'{self.severity} Penalty at {time} P{period}'

In [ ]:
# in certain situtations, the captain of an offending team can elect to play 2 men SH
#    for 2m or 1 man SH for 4m.
# in 5/6 scenarios the captain opted for a double minor. Only the Avs (vs ARI 1/14/22)
# elected to go down 2 men for 2m.

cache_4_PIMs = {
    ((240,), ()): 'DBL',
    ((120,120), ()): '2SH',
    ((540,), (300,)): 'DBL',
    ((240, 120), (120,)): 'CPT',
    ((240, 120, 120), (120, 120)): 'CPT',
    ((360, 300), (300, 120)): 'DBL',
    ((360,), (120,)): 'DBL',
    ((120, 120, 120), (120,)): 'DBL',
    ((420, 120), (300,)): '2SH',
    ((300, 120, 120), (300,)): '2SH',
    ((360, 120), (120, 120)): 'CPT',
    ((300, 240), (300,)): 'DBL'
}

def cache_4_min_PIMS(away_pens, home_pens, time, date):
    more_penalized, less_penalized = None, None
    away_more_penalized = sum(away_pens) > sum(home_pens)
    if away_more_penalized:
        more_penalized = away_pens
        less_penalized = home_pens
    else:
        more_penalized = home_pens
        less_penalized = away_pens
    result = cache_4_PIMs[(more_penalized, less_penalized)]
    assert result, f'Coincidental penalty batch not accounted for: {away_pens}, {home_pens}'
    
    if result == 'CPT':
        result = '2SH' if date == '2022-01-14' else 'DBL'
    
    if result == 'DBL':
        pen = Penalty(time, remaining_minors = 1)
        return ([pen], []) if away_more_penalized else ([], [pen])
    else:
        pen1 = Penalty(time)
        pen2 = Penalty(time)
        return ([pen1, pen2], []) if away_more_penalized else ([], [pen1, pen2])
        

In [ ]:
# 7 minute penalty resolver

# in certain situtations, the captain of an offending team can elect to play 2 men SH
#    (1 for 2m, 1 for 5m), or play 1 man SH for 7m

cache_7_PIMs = {
    ((300, 120), ()): '2SH',
    ((540, 300), (420,)): 'CPT',
    ((420,), ()): '1SH',
}

def cache_7_min_PIMS(away_pens, home_pens, time):
    more_penalized, less_penalized = None, None
    away_more_penalized = sum(away_pens) > sum(home_pens)
    if away_more_penalized:
        more_penalized = away_pens
        less_penalized = home_pens
    else:
        more_penalized = home_pens
        less_penalized = away_pens
    result = cache_7_PIMs[(more_penalized, less_penalized)]
    assert result, f'Coincidental penalty batch not accounted for {away_pens}, {home_pens}'
    if result == 'CPT': # CHI @ NSH 12/1/18
        result = '1SH'
            
    if result == '1SH':
        pen = Penalty(time, 300, 'Major', remaining_minors = 1)
        return ([pen], []) if away_more_penalized else ([], [pen])
    else:
        minor = Penalty(time)
        major = Penalty(time, 300, 'Major')
        # since we call .pop() to add penalties to clock, we must ensure minors are 
        # on the clock first (Rule 26.3)
        return ([major, minor], []) if away_more_penalized else ([], [major, minor])
        

In [ ]:
class Game:
    def __init__(self, away_team, home_team, date, season_type):
        self.away_team = away_team
        self.home_team = home_team
        self.date = date
        self.season_type = season_type # ['R', 'P'], Regular Season or Playoffs
        self.pre_penalties = {} # away/home -> {player_name -> [Penalty]}
        self.pre_penalties['away'] = defaultdict(list) 
        self.pre_penalties['home'] = defaultdict(list)
        self.current_time = 0
        self.period = None
        self.strength = (5,5)

        # records metrics for each team and strength combination for data collection
        self.strength_metrics = {
            'TOI': defaultdict(int), # maps (strength) -> TOI
            'GF': {self.away_team: Counter(), self.home_team: Counter()}, # maps team -> {(strength) -> goals}
            'GA': {self.away_team: Counter(), self.home_team: Counter()} # maps team -> {(strength) -> goals against}
        }
        # contains all penalties that are on the clock (affecting on-ice manpower)
        self.penalty_clock = {}
        self.penalty_clock['away'] = []
        self.penalty_clock['home'] = []

        # penalties that would force a team to have less than 3 skaters on the ice
        # after old ones expire, add queued penalties
        self.queued_penalties = {}
        self.queued_penalties['away'] = []
        self.queued_penalties['home'] = []

        # when penalties expire in OT, teams have an extra skater on the ice until a stoppage
        # i.e. a 4v3 PP ends, play continues as 4v4 until a stoppage
        self.away_OT_extra = 0
        self.home_OT_extra = 0
        # set after stoppages in OT to signify the PP advantage of the home/away team
        # for example if the away team starts OT on the PP, away_OT_PP = 1
        # when the penalty expires, we decrement this and increment both away_OT_extra and home_OT_extra
        self.away_OT_PP = 0
        self.home_OT_PP = 0
        self.regular_season_OT = False
        # for debugging purposes
        self.coincidentals = set([])
        
    # returns the manpower of both teams.
    # since (regular season) overtime penalties work differently than regulation, we need to adjust
    #   the math for OT
    def get_strengths(self):
        if self.period == 4 and self.season_type == 'R': # regular season only
            away_strength = 3 + self.away_OT_extra + self.away_OT_PP
            home_strength = 3 + self.home_OT_extra + self.home_OT_PP

        else: # all other times should be default 5v5 play
            away_strength = 5 - len(self.penalty_clock['away'])
            home_strength = 5 - len(self.penalty_clock['home'])
        return (away_strength, home_strength)
    
    # takes the period and time of a play and returns the time elapsed since the start of the game
    def get_game_seconds(self, period, time):
        return (period - 1) * 1200 + time

    # bulky function that looks at both away/home penalty clocks and removes expired ones
    # if penalties expire and there are queued penalties, we should add them to the clock here
    def update_penalty_clock(self, period, period_seconds, debug_mode = False):
        # get time elapsed since last update
        old_time = self.current_time
        current_seconds = self.get_game_seconds(period, period_seconds)
        secs_elapsed = current_seconds - old_time
        
        # update current time
        self.current_time = current_seconds # period_seconds
        self.period = period
        
        old_strengths = self.get_strengths()

        expired_penalties = {}
        new_penalty_clock = {}
        for _team in ['away', 'home']:
            # update respective penalty clock
            expired_penalties[_team] = []
            new_penalty_clock[_team] = []
            for pen in self.penalty_clock[_team]:
                curr_expiration = pen.time_start + pen.time_length
                if curr_expiration > self.current_time:
                    new_penalty_clock[_team].append(pen)
                else: # if current penalty expired, see if any are remaining
                    if pen.remaining_majors:
                        new_pen_start = min(pen.time_start + 300, self.current_time)
                        rem_majors = pen.remaining_majors - 1
                        new_pen = Penalty(new_pen_start, 500, 'Major', 
                                        remaining_majors = rem_majors, remaining_minors = pen.remaining_minors)
                        new_penalty_clock[_team].append(new_pen)
                    elif pen.remaining_minors:
                        new_pen_start = min(pen.time_start + 120, self.current_time)
                        rem_minors = pen.remaining_minors - 1
                        new_pen = Penalty(new_pen_start, remaining_minors = rem_minors)
                        new_penalty_clock[_team].append(new_pen)
                    else: # keep the expired penalties in case there are queued penalties and we can adjust time_start accordingly
                        expired_penalties[_team].append(pen)
                        
            num_added = 0 # amount of penalties that were queued and have been added to the clock
            # if there is room for another penalty, we should check if there are queued penalties and add them if possible
            if len(new_penalty_clock[_team]) == 1: 
                if expired_penalties[_team] and self.queued_penalties[_team]: # if one expired and one is on deck, add one
                    first_expired = min(expired_penalties[_team], key = lambda pen: pen.time_start + pen.time_length)
                    new_pen = self.queued_penalties[_team].pop()
                    new_pen.time_start = first_expired.time_start + first_expired.time_length
                    new_penalty_clock[_team].append(new_pen)
                    num_added = 1
            elif len(new_penalty_clock[_team]) == 0:
                if expired_penalties[_team] and self.queued_penalties[_team]:
                    if len(self.queued_penalties[_team]) == 2: # if there are 2 queued penalties, we know that 2 just expired
                        exp_pen_1 = expired_penalties[_team][0]
                        new_pen_1 = self.queued_penalties[_team].pop()
                        new_pen_1.time_start = exp_pen_1.time_start + exp_pen_1.time_length
                        new_penalty_clock[_team].append(new_pen_1)
                        
                        exp_pen_2 = expired_penalties[_team][1]
                        new_pen_2 = self.queued_penalties[_team].pop()
                        new_pen_2.time_start = exp_pen_2.time_start + exp_pen_2.time_length
                        new_penalty_clock[_team].append(new_pen_2)
                        num_added = 2
                    elif len(self.queued_penalties[_team]) == 1:
                        first_expired = min(expired_penalties[_team], key = lambda pen: pen.time_start + pen.time_length)
                        new_pen = self.queued_penalties[_team].pop()
                        new_pen.time_start = first_expired.time_start + first_expired.time_length
                        new_penalty_clock[_team].append(new_pen)
                        num_added = 1
            
            num_expired = len(self.penalty_clock[_team]) - len(new_penalty_clock[_team])
            if num_expired and self.regular_season_OT:
                if _team == 'away':
                    self.away_OT_extra += num_expired
                    self.home_OT_extra += num_expired
                    self.home_OT_PP -= num_expired
                else:
                    self.home_OT_extra += num_expired
                    self.away_OT_extra += num_expired
                    self.away_OT_PP -= num_expired
            
            self.penalty_clock[_team] = new_penalty_clock[_team]
            expired_penalties[_team] = expired_penalties[_team][num_added:] # for the strength metrics
        
        self.strength = self.get_strengths()
        
        # exclude shootouts, which mess up the data
        if not (self.season_type == 'R' and self.period > 4): 
            self.update_strength_metrics(expired_penalties['away'], expired_penalties['home'], old_strengths, secs_elapsed, debug_mode)
            
        if debug_mode:
            print(f'Updating Penalty Clock at P:{period} {period_seconds}s')
            print(f'Away: {self.penalty_clock["away"]}')
            print(f'Home: {self.penalty_clock["home"]}')
            print(self.get_strengths())
            
    # update our strength tracking metrics by looking at expired penalties
    # if pens expired here, we know no goal was scored (unless major/match)
    def update_strength_metrics(self, away_expired_penalties, home_expired_penalties, old_strengths, secs_elapsed, debug_mode):
        away_expired = len(away_expired_penalties)
        home_expired = len(home_expired_penalties)
        if away_expired + home_expired == 1:
            if away_expired:
                pen = away_expired_penalties[0]
            else:
                pen = home_expired_penalties[0]
            old_strength_time = self.current_time - pen.time_length - pen.time_start
            self.strength_metrics['TOI'][old_strengths] += secs_elapsed - old_strength_time
            self.strength_metrics['TOI'][self.strength] += old_strength_time
        elif not(away_expired or home_expired):
            self.strength_metrics['TOI'][old_strengths] += secs_elapsed
        else: # complex case
            # maps expiration_time -> <'away'/'home'> to indicate who the expiring penalty was on
            expired_pens = defaultdict(list) 
            expiration_times = set([]) # stores unique penalty expiration times
            
            for away_pen in away_expired_penalties:
                expiration = away_pen.time_start + away_pen.time_length
                expiration_times.add(expiration)
                expired_pens[expiration].append('away')
                
            for home_pen in home_expired_penalties:
                expiration = home_pen.time_start + home_pen.time_length
                expiration_times.add(expiration)
                expired_pens[expiration].append('home')
                
            # sort in ascending order of expiration
            expiration_times = sorted(list(expiration_times))
            
            new_strengths = old_strengths
            # keeps track of time in the past as we are retroactively inferring TOI for each manpower
            checkpoint = self.current_time - secs_elapsed
            for curr_exp_time in expiration_times:
                curr_expired_pens = expired_pens[curr_exp_time]
                time_diff = curr_exp_time - checkpoint # time that given manpower was on ice
                
                self.strength_metrics['TOI'][new_strengths] += time_diff
                checkpoint += time_diff
                # for each penalty that expires at exactly this time, increment the manpower
                for team in curr_expired_pens:
                    if team == 'away':
                        new_strengths = (new_strengths[0] + 1, new_strengths[1])
                    else:
                        new_strengths = (new_strengths[0], new_strengths[1] + 1)
                        
            # don't forget to increment the current manpower TOI with the remaining time
            self.strength_metrics['TOI'][new_strengths] += self.current_time - checkpoint
        if debug_mode:
            print(self.strength_metrics['TOI'])
    
    # sets the current strength of the Game object
    def update_strength(self):
        self.strength = self.get_strengths()     
    
    # this helper function is called when a PPG is scored to remove the first (non-4v4) minor to expire
    def get_expiring_penalty(self, team_penalty_clock):
        first_expiration = float('inf')
        first_expiration_pen = None
        for i, pen in enumerate(team_penalty_clock):
            curr_expiration = pen.time_start + pen.time_length
            if pen.severity == 'Minor' and not pen.is_4v4 and curr_expiration < first_expiration:
                first_expiration_pen = pen
                first_expiration = curr_expiration
        return first_expiration_pen
    
    # - helper function called after a PPG
    # - we look at the SH team's penalty clock and remove the first non-coincidental minor set to expire
    # - additionally, if we remove a minor penalty and there are queued penalties, we should add them here until there
    #     are 2 penalties on the clock, or there are no queued penalties.
    def remove_minor(self, team):
        if team == self.away_team: 
            expiring_penalty = self.get_expiring_penalty(self.penalty_clock['away'])
            if expiring_penalty:
                self.penalty_clock['away'].remove(expiring_penalty)
                if expiring_penalty.remaining_minors > 0:
                    rem_minors = expiring_penalty.remaining_minors - 1
                    new_pen = Penalty(self.current_time, remaining_minors = rem_minors)
                    self.penalty_clock['away'].append(new_pen)
                elif self.queued_penalties['away']: # check for queued penalties
                    new_pen = self.queued_penalties['away'].pop()
                    new_pen.time_start = self.current_time
                    self.penalty_clock['away'].append(new_pen)
        else:
            expiring_penalty = self.get_expiring_penalty(self.penalty_clock['home'])
            if expiring_penalty:
                self.penalty_clock['home'].remove(expiring_penalty)
                if expiring_penalty.remaining_minors > 0:
                    rem_minors = expiring_penalty.remaining_minors - 1
                    new_pen = Penalty(self.current_time, remaining_minors = rem_minors)
                    self.penalty_clock['home'].append(new_pen)
                elif self.queued_penalties['home']: # check for queued penalties
                    new_pen = self.queued_penalties['home'].pop()
                    new_pen.time_start = self.current_time
                    self.penalty_clock['home'].append(new_pen)
            
    # - this function is called when a goal is scored
    # - we first call update_penalty_clock() to remove penalties that may have expired due to time
    # - we then consider the on-ice manpower and if the scoring team was on the PP, remove a minor 
    #     penalty from the other team if possible.
    def add_goal(self, play, debug_mode = False):
        period_goal_scored = play['period']
        time_goal_scored = play['period_seconds']
        
        # remove penalties that have expired
        self.update_penalty_clock(period_goal_scored, time_goal_scored, debug_mode)
        
        away_strength, home_strength = self.get_strengths()
        if debug_mode:
            print(f'GOAL!! {away_strength}v{home_strength}')
            
        # Rule 16.2: no penalty shall expire due to a penalty shot goal
        if play['secondary_type'] == 'Penalty Shot':
            return
        
        # add goals to the given strength in self.strength_to_goals
        self.update_goal_metric(play)
        
        if away_strength != home_strength: # no penalty shall expire at even strength
            # remove any applicable minor penalties on the clock, iff scoring team was on PP
            team_scored = play['event_team']
            if team_scored == self.away_team and away_strength > home_strength:
                self.remove_minor(self.home_team)
            elif team_scored == self.home_team and home_strength > away_strength:
                self.remove_minor(self.away_team)
            
        self.update_strength()
        
        if debug_mode:
            print('After Goal:')
            print(f'Away: {self.penalty_clock["away"]}')
            print(f'Home: {self.penalty_clock["home"]}')
    
    # - used for data scraping purposes
    # - after a goal is scored, we increment the respective teams' GF and GA at the given strength
    def update_goal_metric(self, play):
        scoring_team = play['event_team']
        away_strength, home_strength = self.get_strengths()
        if scoring_team == self.away_team:
            self.strength_metrics['GF'][scoring_team][(away_strength, home_strength)] += 1
            self.strength_metrics['GA'][self.home_team][(home_strength, away_strength)] += 1
        elif scoring_team == self.home_team:
            self.strength_metrics['GF'][scoring_team][(home_strength, away_strength)] += 1
            self.strength_metrics['GA'][self.away_team][(away_strength, home_strength)] += 1

    # - add a penalty to the current batch of penalties
    # - we use the PrePenalty class as these are being preprocessed
    # - we really only care about the possibility of 5+2s, 2+2s, etc; and the total PIMs
    def add_penalty(self, play):
        penalized_team = play['event_team']
        severity = play['penalty_severity']
            
        penalty = PrePenalty()
        if severity == 'Minor': # check PIMs if 2 or 4
            if play['penalty_minutes'] == 4: # double minor is 2 minors back to back, not a single 4 minute block
                penalty.time_length = 240
            elif play['penalty_minutes'] == 6: # triple minor
                penalty.time_length = 360
        elif severity == 'Major' or severity == 'Match': # 5 minutes
            penalty.time_length = 300

        penalized_player = play['event_player_1_name']
        if penalized_team == self.away_team:
            self.pre_penalties['away'][penalized_player].append(penalty)
        else:
            self.pre_penalties['home'][penalized_player].append(penalty)

    # - once the current batch of penalties is completed, determine the on ice strength
    #    using self.<away/home>_penalty_clock and try to resolve coincidental penalties
    # - once we match coincidental penalties, only add the leftover penalties on the clock
    # - if the penalty clock is full, add the rest of them to self.queued_<away/home>_penalties
    def process_penalties(self, play, debug_mode = False):
        # group penalties by player. if they have multiple, add the PIMs and we can infer them
        away_pens = self.get_team_pens(self.pre_penalties['away'])
        home_pens = self.get_team_pens(self.pre_penalties['home'])
        
        # clear expired penalties and keep self.current_time updated
        self.update_penalty_clock(play['period'], play['period_seconds'])
        
        away_PIMs = sum(away_pens)
        home_PIMs = sum(home_pens)
        PIMs_diff = away_PIMs - home_PIMs
        abs_PIMs_diff = abs(PIMs_diff)
        
        away_team_penalized = PIMs_diff > 0
        penalized_team = self.away_team if away_team_penalized else self.home_team
        
        # we keep temporary lists of the penalties to be added in case we need to queue one or more
        # i.e. if a team has 1 on the clock and takes 2 more penalties, we can only add 1 and need to queue the other
        away_temp_pens = []
        home_temp_pens = []
        
        # we can use some clever logic by taking the difference between the two teams' PIMs to deduce what
        # most penalty combinations will result. 
        # i.e. - 120 will always result in 1 team with a 2m minor
        #      - 180 will always result in 1 team with a 5m major and the other team with a 2m minor
        # note that a few PIMs_diff are ambiguous and that we need to resolve them, like 240 or 420
        match abs_PIMs_diff:
            case 120: # 2m PP
                pen = Penalty(self.current_time)
                away_temp_pens.append(pen) if away_team_penalized else home_temp_pens.append(pen)
            case 300: # 5m PP
                pen = Penalty(self.current_time, 300, 'Major')
                away_temp_pens.append(pen) if away_team_penalized else home_temp_pens.append(pen)
            case 0: # Full strength unless both teams have exactly 1 2m minor
                if away_pens == (120,) and home_pens == (120,) and self.get_strengths() == (5,5):
                    pen1 = Penalty(self.current_time, is_4v4 = True)
                    pen2 = Penalty(self.current_time, is_4v4 = True)
                    away_temp_pens.append(pen1)
                    home_temp_pens.append(pen2)
            case 240:
                away_temp_pens, home_temp_pens = cache_4_min_PIMS(away_pens, home_pens, self.current_time, play['game_date'])
            case 60: # 5 vs 2, 2
                print('(debug) PIM difference == 1; Assuming 1 major vs 2 minors')
                print(f'(debug) {away_pens}, {home_pens}')
                minor1 = Penalty(self.current_time)
                minor2 = Penalty(self.current_time)
                major = Penalty(self.current_time, 500, 'Major')
                if self.current_time >= 3300:
                    print(f'L5M/OT: {away_pens}, {home_pens}')
                    assert False, f"L5M/OT situation {self.away_team} @ {self.home_team} {self.date}"
                else:
                    if away_team_penalized: # assign the major to the away team
                        away_temp_pens.append(major)
                        home_temp_pens = [minor1, minor2]
                    else: # assign the major to the home team
                        away_temp_pens = [minor1, minor2]
                        home_temp_pens.append(major)
            case 180: # 5 vs 2
                # rule 19.4 for last 5 mins and overtime
                if self.current_time >= 3300:
                    print(f'(debug) L5M/OT: {away_pens}, {home_pens}')
                    print(f'(debug) {self.away_team} @ {self.home_team} {self.date}')
                    major = Penalty(self.current_time, 180, 'Major')
                    away_temp_pens.append(major) if away_team_penalized else home_temp_pens.append(major)
                else:
                    major = Penalty(self.current_time, 300, 'Major')
                    minor = Penalty(self.current_time)
                    if away_team_penalized: # assign the major to the away team
                        away_temp_pens.append(major)
                        home_temp_pens.append(minor)
                    else: # assign the major to the home team
                        away_temp_pens.append(minor)
                        home_temp_pens.append(major)
            case 360: # 2 + 2 + 2
                print('(debug) PIM difference == 6; Assuming a 2+2+2')
                print(f'(debug) {self.away_team} @ {self.home_team} {self.date}')
                print(away_pens, home_pens)
                pen = Penalty(self.current_time, remaining_minors = 2)
                away_temp_pens.append(pen) if away_team_penalized else home_temp_pens.append(pen)
            case 420: # 5 + 2
                if debug_mode:
                    print(away_pens, home_pens)
                away_temp_pens, home_temp_pens = cache_7_min_PIMS(away_pens, home_pens, self.current_time)
            case 540: # 5 + 2 + 2
                print('(debug) PIM difference == 9; Assuming a 5+2+2')
                print(f'(debug) {self.away_team} @ {self.home_team} {self.date}')
                print(away_pens, home_pens)
                pen = Penalty(self.current_time, 300, 'Major', remaining_minors = 2)
                away_temp_pens.append(pen) if away_team_penalized else home_temp_pens.append(pen)
            case 600: # 5 + 5, 5 // 5
                print('(debug) PIM difference == 9; Assuming a 5+5')
                print(f'(debug) {self.away_team} @ {self.home_team} {self.date}')
                print(away_pens, home_pens)
                pen = Penalty(self.current_time, 300, 'Major', remaining_majors = 1)
                away_temp_pens.append(pen) if away_team_penalized else home_temp_pens.append(pen)
            case default: # disregard for now
                print(abs_PIMs_diff)
                print('(debug) Default case for PIM matching')
                print(f'(debug) {self.away_team} @ {self.home_team} {self.date}')
                print(away_pens, home_pens)
                assert False, "Current batch of coincidental penalties not accounted for: {away_pens}, {home_pens}"
                
        temp_pens = {'away': away_temp_pens, 'home': home_temp_pens}
        self.add_penalties_to_clock(temp_pens)
        
        if self.regular_season_OT:
            self.reset_OT_strengths()
            
        self.update_strength()

        # reset pre-processed penalties
        self.pre_penalties['away'] = defaultdict(list)
        self.pre_penalties['home'] = defaultdict(list)
                
        if debug_mode:
            print('Processed penalties. Resulting Penalty Clock: ')
            print(f'PIMs_diff: {PIMs_diff}')
            print(f'Away: {self.penalty_clock["away"]}')
            print(f'Home: {self.penalty_clock["home"]}')
        
    # - helper function that takes all penalties drawn and attempts to add them to the penalty clock
    # - the penalty clock cannot have more than 2 penalties, so we add the rest to the queue 
    # - the queued penalties will begin once earlier ones expire
    def add_penalties_to_clock(self, temp_pens):
        for _team in ['away', 'home']:
            # see if we can add these penalties to the clock, else, queue them
            if len(self.penalty_clock[_team]) == 2: # penalty clock full, queue the current batch
                self.queued_penalties[_team] += temp_pens[_team]
            elif len(self.penalty_clock[_team]) == 1: # can add 1 penalty to the clock
                if temp_pens[_team]:
                    pen = temp_pens[_team].pop()
                    self.penalty_clock[_team].append(pen)
                    self.queued_penalties[_team] += temp_pens[_team]
            elif len(self.penalty_clock[_team]) == 0: # penalty clock empty, add up to 2 from the batch
                if len(temp_pens[_team]) > 2:
                    pen1 = temp_pens[_team].pop()
                    pen2 = temp_pens[_team].pop()
                    self.penalty_clock[_team] = [pen1, pen2]
                    self.queued_penalties[_team] += temp_pens[_team]
                else:
                    self.penalty_clock[_team] = temp_pens[_team]
            else:
                assert False, f"{_team} Penalty Clock is misconfigured"
    
    # team_penalties: player -> [Penalty]
    # - groups total PIMs per player to account for double minors, 5+2s, etc
    # retval: tuple of each players total PIMs
    def get_team_pens(self, team_penalties):
        team_pens = ()
        for player, pens in team_penalties.items():
            curr_PIMs = sum(pen.time_length for pen in pens)
            team_pens = (*team_pens, curr_PIMs)
        return tuple(sorted(list(team_pens), reverse = True))
    
    # clear 4v4 penalties
    # since regular season OT has different rules with penalties on 3v3, we need a different Game state.
    def start_OT(self):
        self.regular_season_OT = True
        self.penalty_clock['away'] = [pen for pen in self.penalty_clock['away'] if not pen.is_4v4]
        assert len(self.queued_penalties['away']) == 0, f"There are queued penalties heading to OT {self.away_team} v {self.home_team} {self.date}"
        self.penalty_clock['home'] = [pen for pen in self.penalty_clock['home'] if not pen.is_4v4]
        assert len(self.queued_penalties['home']) == 0, f"There are queued penalties heading to OT {self.away_team} v {self.home_team} {self.date}"
        self.reset_OT_strengths()
        
    # after any stoppage in regular season OT, reset back to the default 3v3 or set PP fields if there is a PP
    def reset_OT_strengths(self):
        self.away_OT_extra = 0
        self.home_OT_extra = 0
        self.away_OT_PP = 0
        self.home_OT_PP = 0
        OT_PP = len(self.penalty_clock['away']) - len(self.penalty_clock['home'])
        if OT_PP > 0:
            self.home_OT_PP = OT_PP
        elif OT_PP < 0:
            self.away_OT_PP = -OT_PP
    
    # debugging function
    # gets every unique combination of penalties called at a single stoppage
    def increment_coincidentals_debug(self):
        away_pens = self.get_team_pens(self.pre_penalties['away'])
        home_pens = self.get_team_pens(self.pre_penalties['home'])

        if sum(away_pens) > sum(home_pens):
            self.coincidentals.add((away_pens, home_pens))
        else:
            self.coincidentals.add((home_pens, away_pens))

        self.pre_penalties['away'] = defaultdict(list)
        self.pre_penalties['home'] = defaultdict(list)
        
    # designed to be used with the coincidental penalty debugging function
    def add_penalty_debug(self, play):
        penalized_team = play['event_team']
        severity = play['penalty_severity']
        period = play['period']
        time = play['period_seconds']
        self.current_time = time
        self.period = period

        penalty = PrePenalty()
        if severity == 'Minor': # check PIMs if 2 or 4
            PIMs = play['penalty_minutes']
            if PIMs == 4: # double minor is 2 minors back to back, not a single 4 minute block
                penalty.time_length = 240
            elif PIMs == 6: 
                penalty.time_length = 360
                penalty.severity = 'Triple Minor'
        elif severity == 'Major' or severity == 'Match': # 5 minutes
            penalty.time_length = 300

        penalized_player = play['event_player_1_name']
        if penalized_team == self.away_team:
            self.pre_penalties['away'][penalized_player].append(penalty)
        else:
            self.pre_penalties['home'][penalized_player].append(penalty)

In [ ]:
# Upon completion of scraping a single game, this function updates the annual total for each respective team
def update_team_statistics(strength_metrics, current_metrics, year, team, is_home):
    if team in ['Canadian All-Stars', 'American All-Stars']:
        return strength_metrics
    else:
        if is_home:
            # if we are currently processing the home team, we need to reverse the strength notation i.e. 4v5 -> 5v4       
            for _strength, _time in current_metrics['TOI'].items():
                home_strength = (_strength[1], _strength[0])
                strength_metrics['TOI'][year][team][home_strength] += _time

            strength_metrics['GF'][year][team] += current_metrics['GF'][team]
            strength_metrics['GA'][year][team] += current_metrics['GA'][team]
        else:
            strength_metrics['TOI'][year][team] += current_metrics['TOI']
            strength_metrics['GF'][year][team] += current_metrics['GF'][team]
            strength_metrics['GA'][year][team] += current_metrics['GA'][team]

        return strength_metrics

In [ ]:
DTYPES = ['home_division_name_short', 'away_division_name_short']
def scrape_pbp(first_year = 16, last_year = 22, playoffs = False):
    strength_metrics = {}
    for _m in METRICS:
        strength_metrics[_m] = {}

    for year in range(first_year, last_year + 1):
        season = int(f'20{year}')
        for _m in METRICS:
            strength_metrics[_m][season] = {}
            for _team in ALL_TEAMS:
                strength_metrics[_m][season][_team] = Counter()
    
    current_game = None
    penalty_just_called = False

    # these are irrelevant to on-ice manpower and PP goals (misconducts usually paired with a minor/major)
    irrelevant_penalties = set(['Penalty Shot', 'Misconduct', 'Game Misconduct'])
    for year in range(first_year, last_year + 1):
        file = PATH_TO_DATA + 'play_by_play_20' + str(year) + '_' + str(year+1) + '_lite.csv'

        encoding = None if year == 22 else "latin-1"
        dtypes = {"home_on_7": "string", "away_on_7": "string"}
        if year < 18:
            dtypes = dtypes | {'home_division_name_short': "string", 'away_division_name_short': "string"}

        curr_pbp = pd.read_csv(file, encoding=encoding, dtype=dtypes)

        print(f"Scraping the 20{year}-20{year+1} season...")
        for _, play in tqdm(curr_pbp.iterrows()):
            if playoffs and play['season_type'] == 'R':
                continue
            else:
                event = play['event_type']

                away_team, home_team, date = play['away_name'], play['home_name'], play['game_date']
                if current_game: # sometimes, games don't have the 'GAME_SCHEDULED' event, need to manually check
                    if away_team != current_game.away_team or home_team != current_game.home_team or date != current_game.date:
                        strength_metrics = update_team_statistics(strength_metrics, current_game.strength_metrics, int(f'20{year}'), current_game.away_team, False)
                        strength_metrics = update_team_statistics(strength_metrics, current_game.strength_metrics, int(f'20{year}'), current_game.home_team, True)
                        
                        current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])

                if penalty_just_called and event != 'PENALTY': # once all penalties are announced, process them and figure out the manpower
                    current_game.process_penalties(play)
                    penalty_just_called = False

                if play['period'] == 4 and current_game.season_type == 'R':
                    if not current_game.regular_season_OT:
                        current_game.start_OT()
                    elif play['event_type'] in set(['STOP', 'CHALLENGE', 'PENALTY']):
                        current_game.reset_OT_strengths()

                match event:
                    case 'GAME_SCHEDULED':
                        if current_game:
                            strength_metrics = update_team_statistics(strength_metrics, current_game.strength_metrics, int(f'20{year}'), current_game.away_team, False)
                            strength_metrics = update_team_statistics(strength_metrics, current_game.strength_metrics, int(f'20{year}'), current_game.home_team, True)
                            
                        current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])
                    case 'PENALTY':
                        if play['penalty_severity'] not in irrelevant_penalties:  # minor/major/misconduct/game misconduct/match
                            current_game.add_penalty(play)
                            penalty_just_called = True
                    case 'GOAL': # exclude penalty shots and shootout goals
                        if play['secondary_type'] != 'Penalty Shot' and play['period'] != 5: # exclude SO
                            current_game.add_goal(play)
                    case default:
                        current_game.update_penalty_clock(play['period'], play['period_seconds'])

    print('Done scraping...')
    return strength_metrics

# scrape_pbp(False, 16, 16)

In [131]:
def collect_data(first_year, last_year, playoffs = False):
    assert first_year < 23 and first_year > 15 and type(first_year) == int, "year parameters must be ints between 16 and 22"
    assert last_year < 23 and last_year > 15 and type(last_year) == int, "year parameters must be ints between 16 and 22"
    assert first_year <= last_year, "first year must be less than or equal to last year"
    strength_metrics = scrape_pbp(first_year, last_year, playoffs)

    print("Beginning to export the data to a dataframe...")
    data = []
    for year in range(2000 + first_year, 2000 + last_year + 1):
        teams = strength_metrics['TOI'][year].keys()
        for _team in teams:
            curr_row = [year, _team]
            for _strength in STRENGTHS:
                curr_toi = strength_metrics['TOI'][year][_team][_strength]
                curr_goals = strength_metrics['GF'][year][_team][_strength]
                curr_goals_against = strength_metrics['GA'][year][_team][_strength]

                new_row = curr_row + [_strength, curr_toi, curr_goals, curr_goals_against]
                data.append(new_row)

    df = pd.DataFrame(data, columns=COLUMNS)
    output_name = f'./data/nhl_strength_data_20{first_year}_{last_year+1}'
    suffix = "_playoffs.csv" if playoffs else ".csv"
    output_name += suffix
    df.to_csv(output_name)
    print(f"Done exporting the data to: {output_name}")

In [134]:
collect_data(16, 21, True)

KeyboardInterrupt: 

In [ ]:
# - this function has a similar body scrape_pbp() but is used for debugging
# - specficially, when testing new or edited features, you can uncomment the first 2 commented lines to find the 
#   game number (i)
# - once you have the game number, insert this into the next commented 2 lines and the scraper will enter debug mode,
#   printing the game state on a play-by-play basis

# Example: you know that BOS @ CBJ has a triple minor and want to ensure the system is handling this properly
# 1. add the teams to the first if statement (say this returns i=238)
# 2. set game_num to 238 and uncomment the 2nd pair of lines
# 3. the function will print out the relevant features on a pbp basis, which can be cross 
#    referenced (ESPN pbp) and verified
def scrape_pbp_debug():
    team_to_power_plays = {}
    current_game = None
    penalty_just_called = False

    # these are irrelevant to on-ice manpower and PP goals (misconducts usually paired with a minor/major)
    irrelevant_penalties = set(['Penalty Shot', 'Misconduct', 'Game Misconduct'])
    
    # debugging variables
    i = 0
    game_num = 118
    debug = False

    pbp_debug = pd.read_csv(PATH_TO_DATA + "play_by_play_2018_19_lite.csv", encoding="latin-1")
    for _, play in tqdm(pbp_debug.iterrows()):
        event = play['event_type']

        away_team, home_team, date = play['away_name'], play['home_name'], play['game_date']
        if current_game:
            if away_team != current_game.away_team or home_team != current_game.home_team or date != current_game.date:
                current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])

        if penalty_just_called and event != 'PENALTY': # once all penalties are announced, process them and figure out the manpower
            current_game.process_penalties(play, debug)
            penalty_just_called = False

        if play['period'] == 4 and current_game.season_type == 'R':
            if not current_game.regular_season_OT:
                current_game.start_OT()
            elif play['event_type'] in set(['STOP', 'CHALLENGE', 'PENALTY']):
                current_game.reset_OT_strengths()

        match event:
            case 'GAME_SCHEDULED':
                current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])
#                 if current_game.away_team == 'Columbus Blue Jackets' and current_game.home_team == 'Boston Bruins':
#                     print(f'bos @ cbj i={i}')
                if current_game.away_team == 'Chicago Blackhawks' and current_game.home_team == 'Arizona Coyotes':
                    print(f'chi @ ari i={i}')
#                 if i == game_num: # debugging
#                     print(f'New Game: {current_game.away_team} @ {current_game.home_team}')
            case 'PENALTY':
                if play['penalty_severity'] not in irrelevant_penalties:  # minor/major/misconduct/game misconduct/match
                    current_game.add_penalty(play)
                    penalty_just_called = True
            case 'GOAL': # exclude penalty shots and shootout goals
                if play['secondary_type'] != 'Penalty Shot' and play['period'] != 5: # exclude SO
                    current_game.add_goal(play, debug)
            case 'GAME_END':
                # debugging 
                i += 1
                # if i > game_num - 1:
                #     debug = True
                # if i > game_num:
                #     break # debugging
            case default:
                current_game.update_penalty_clock(play['period'], play['period_seconds'], debug)

    print('Done')
    
scrape_pbp_debug()

In [ ]:
# - this function was used to scrape most pbp years and gather the unique combinations of penalties called
#   at a single stoppage.
# - after gathering and printing the combinations, we use the rulebook to determine what the resulting
#   penalties should be after cancelling coincidental penalties.
# - Note: there are ambiguous situations where the penalized team's captain has the choice to, for example,
#   go SH 2 men for 2 minutes or SH 1 man for 4 minutes.
def get_coincidental_penalties(first_year = 16, last_year = 22):
    current_game = None
    penalty_just_called = False
    PIM_to_coincidental = defaultdict(set)
    coincidentals = set([])

    # these are irrelevant to on-ice manpower and PP goals (misconducts usually paired with a minor/major)
    irrelevant_penalties = set(['Penalty Shot', 'Misconduct', 'Game Misconduct'])

    for year in range(first_year, last_year + 1):
        pbp_test = pd.read_csv(PATH_TO_DATA + f"play_by_play_20{year}_{year+1}_lite.csv", encoding='latin-1')
        for _, play in tqdm(pbp_test.iterrows()):
            event = play['event_type']

            away_team, home_team, date = play['away_name'], play['home_name'], play['game_date']
            if current_game:
                if away_team != current_game.away_team or home_team != current_game.home_team or date != current_game.date:
                    current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])

            match event:
                case 'GAME_SCHEDULED':
                    current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])
                case 'PENALTY':
                    if play['penalty_severity'] not in irrelevant_penalties:  # minor/major/misconduct/game misconduct/match
                        current_game.add_penalty_debug(play)
                        penalty_just_called = True
                case 'GAME_END':
                    coincidentals.update(current_game.coincidentals)

            if penalty_just_called and event != 'PENALTY': # once all penalties are announced, process them and figure out the manpower
                current_game.increment_coincidentals_debug()
                penalty_just_called = False

    for combo in coincidentals:
        t1, t2 = combo
        diff = sum(t1) - sum(t2)
        PIM_to_coincidental[diff].add(combo)

    return PIM_to_coincidental

In [ ]:
def print_coincidental_penalties():
    first_year = 16
    last_year = 17
    cache = get_coincidental_penalties(first_year, last_year)
    print(cache.keys()) # outline the unique differences in PIMs
    del cache[0]

    for k, v in cache.items():
        if k not in [120, 300]: # these are trivial cases
            print(f'PIM difference == {k}')
            for combo in v:
                print(combo)

In [ ]:
print_coincidental_penalties()